- python 2.7  
- cv2 3.4

In [1]:
import numpy as np
import cv2
from cv2 import aruco
import pickle
import glob
import math
import os 
import matplotlib.pyplot as plt
import matplotlib as mpl
# import pandas as pd
# %matplotlib nbagg
%matplotlib inline

In [2]:
def save_img(name, img):
    file_name = os.path.join("output/tem", name)
    cv2.imwrite(file_name, img)

In [3]:
def match_template(image, template):
    image_draw = image.copy()
    
    if len(image.shape) == 3 and image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    if len(template.shape) == 3 and template.shape[2] == 3:
        template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    
    
    # 获取模板的尺寸
    w, h = template.shape[::-1]

    # 进行模板匹配
    res = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

    # 找到匹配数值最高的位置
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    # 打印出最高匹配值的位置
    # print("最高匹配值位置:", max_loc)
    # print(res[max_loc[1], max_loc[0]])
    max_value = res[max_loc[1], max_loc[0]]

    # 在原图像中绘制矩形框标记匹配位置
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    
    # 检查图像是否是彩色图像
    if len(image_draw.shape) < 3 or image_draw.shape[2] == 1:
        # print("图像是灰度图，将其转换为彩色图像。")
        image_draw = cv2.cvtColor(image_draw, cv2.COLOR_GRAY2BGR)
    # else:
    #     print("图像是彩色图像。")
    
    
    cv2.rectangle(image_draw, top_left, bottom_right, (0, 255, 0), 2)
    
    return image_draw, max_value

In [4]:
def write_text_on_image(image, max_value, org=(50, 50)):    

    # 定义多行文字
    lines_of_text = [
        f"template matching: {str(max_value)}",
    ]
    # 定义初始位置（左下角）
    org = org
    
    # 定义字体
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    # 定义字体大小
    font_scale = 1
    
    # 定义背景颜色（BGR格式）
    bg_color = (0, 0, 0)  # 黑色
    
    # 定义字体厚度
    thickness = 2
    
    # 定义行间距
    line_spacing = 15
    
    text_color = (255, 255, 255)

    # 在图片上添加多行带背景色的文字
    for i, line in enumerate(lines_of_text):
        # 计算文字大小
        (text_width, text_height), baseline = cv2.getTextSize(line, font, font_scale, thickness)
        
        # 计算每行文字的位置
        x, y = org[0], org[1] + i * (text_height + line_spacing)
        
        # 绘制背景矩形
        cv2.rectangle(image, (x, y - text_height - baseline), (x + text_width, y + baseline), bg_color, -1)

        cv2.putText(image, line, (x, y), font, font_scale, text_color, thickness, cv2.LINE_AA)

    
    return image

__name__ == __main__

In [5]:
import pyrealsense2 as rs
import numpy as np
import cv2
from datetime import datetime
import os

p = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
# config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
# config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
p.start(config)

imu_p = rs.pipeline()
imu_config = rs.config()
imu_config.enable_stream(rs.stream.accel)
imu_p.start(imu_config)

# path
current_time = datetime.now().strftime("%b%d_H%H_M%M_S%S")

template = cv2.imread(r"C:\Users\NEAF\code\yen\HandEye\output\tem\tem.bmp", 0)

template = template[100:400, 950:1250]
try:
    while True:
        frames = p.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        depth_image = cv2.flip(depth_image, -1)
        color_image = cv2.flip(color_image, -1)  
        color_image__ = color_image.copy()      
        
        color_image, max_value = match_template(color_image, template)          
                  
        color_image = write_text_on_image(color_image, round(max_value, 3))
            
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        images = np.hstack((color_image, depth_colormap))

        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', color_image)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
        elif key & 0xFF == ord('s'):
            f = open('acc_rot_tran.pckl', 'wb')
            pickle.dump((accel, rvec2, tvec2), f)
            f.close()            
        elif key & 0xFF == ord('t'):
            save_img("tem.bmp", color_image__)    
            template = cv2.cvtColor(color_image__, cv2.COLOR_BGR2GRAY)
            template = template[100:400, 950:1250]
               

finally:
    p.stop()
    imu_p.stop()

In [6]:
# plt.imshow(cv2.cvtColor(color_image__, cv2.COLOR_BGR2RGB))
# # plt.imshow(cv2.cvtColor(write_text_on_image(color_image__, max_value), cv2.COLOR_BGR2RGB))
# # write_text_on_image(color_image, max_value)

In [7]:
# iii, txt = write_text_on_image(color_image, max_value)